# Load dataset from Google Drive

In [1]:
import gdown

url = 'https://drive.google.com/uc?id=1cwQ3frGJiUEJ1mdCt9GXbIxMDb04hDA2'
output = 'dataset.zip'
gdown.download(url, output, quiet=False)

import zipfile
import os

zip_path = '/content/dataset.zip'
extract_path = '/content'

if not os.path.exists(extract_path):
    os.makedirs(extract_path)

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

Downloading...
From (original): https://drive.google.com/uc?id=1cwQ3frGJiUEJ1mdCt9GXbIxMDb04hDA2
From (redirected): https://drive.google.com/uc?id=1cwQ3frGJiUEJ1mdCt9GXbIxMDb04hDA2&confirm=t&uuid=d06f3986-3b98-4bd9-a2e7-56a3762a5f41
To: /content/dataset.zip
100%|██████████| 624M/624M [00:11<00:00, 52.0MB/s]


# Resize 224x224

In [2]:
!pip install opencv-python

import cv2
import os

def resize_images(input_dir, output_dir, size=(224, 224)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for root, _, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(root, file)
                img = cv2.imread(img_path)
                img_resized = cv2.resize(img, size)
                relative_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                if not os.path.exists(output_subdir):
                    os.makedirs(output_subdir)
                cv2.imwrite(os.path.join(output_subdir, file), img_resized)

input_dir = '/content/combined_dataset'
output_dir = '/content/resized_dataset'
resize_images(input_dir, output_dir)

# Split 80% train, 10% validation, 10% test

In [3]:
import os
import shutil
import random

combined_dir = "/content/combined_dataset"
split_base = "/content/dataset_split"

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

splits = ["train", "val", "test"]

for split in splits:
    split_dir = os.path.join(split_base, split)
    os.makedirs(split_dir, exist_ok=True)

# Iterate through each label in the combined_dataset and split the data
for label in os.listdir(combined_dir):
    label_path = os.path.join(combined_dir, label)
    if not os.path.isdir(label_path):
        continue

    # Get a list of all images in the label folder
    images = [f for f in os.listdir(label_path) if f.endswith(('.jpg', '.jpeg', '.png'))]
    total_images = len(images)

    if total_images == 0:
        print(f"No image in {label}, skipping...")
        continue

    # Shuffle image list
    random.shuffle(images)

    train_count = int(total_images * train_ratio)
    val_count = int(total_images * val_ratio)
    test_count = total_images - train_count - val_count

    train_images = images[:train_count]
    val_images = images[train_count:train_count + val_count]
    test_images = images[train_count + val_count:]

    # Copy images to the corresponding folders
    for split, split_images in zip(splits, [train_images, val_images, test_images]):
        split_label_dir = os.path.join(split_base, split, label)
        os.makedirs(split_label_dir, exist_ok=True)

        for img_name in split_images:
            src = os.path.join(label_path, img_name)
            dst = os.path.join(split_label_dir, img_name)
            shutil.copy(src, dst)

    print(f"\nLabel: {label}")
    print(f"Train: {len(train_images)} images")
    print(f"Val: {len(val_images)} images")
    print(f"Test: {len(test_images)} images")

print(f"\nSuccessfullt splitting data at: {split_base}")


Label: Metal_Other
Train: 2400 images
Val: 300 images
Test: 300 images

Label: Glass
Train: 2400 images
Val: 300 images
Test: 300 images

Label: Organic_Food_Waste
Train: 2400 images
Val: 300 images
Test: 300 images

Label: Paper_Cardboard
Train: 2400 images
Val: 300 images
Test: 300 images

Label: Plastic
Train: 2400 images
Val: 300 images
Test: 300 images

Successfullt splitting data at: /content/dataset_split


# Training with YOLOv8cls

In [ ]:
!pip install ultralytics

import torch
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("no GPU")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 67.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n-cls.pt')

model.train(
    data='/content/dataset_split',
    epochs=30,
    imgsz=224,
    batch=32,
    project='my_yolo_project',
    name='waste_classifier'
)


Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=32, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/dataset_split, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=waste_classifier2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0

train: Scanning /content/dataset_split/train... 12000 images, 0 corrupt: 100%|██████████| 12000/12000 [00:00<?, ?it/s]


val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 577.5±373.5 MB/s, size: 30.8 KB)


val: Scanning /content/dataset_split/val... 1500 images, 0 corrupt: 100%|██████████| 1500/1500 [00:00<?, ?it/s]


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 26 weight(decay=0.0), 27 weight(decay=0.0005), 27 bias(decay=0.0)
Image sizes 224 train, 224 val
Using 2 dataloader workers
Logging results to my_yolo_project/waste_classifier2
Starting training for 30 epochs...

      Epoch    GPU_mem       loss  Instances       Size


...

                   all      0.975          1



30 epochs completed in 0.625 hours.
Optimizer stripped from my_yolo_project/waste_classifier2/weights/last.pt, 3.0MB
Optimizer stripped from my_yolo_project/waste_classifier2/weights/best.pt, 3.0MB

Validating my_yolo_project/waste_classifier2/weights/best.pt...
Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,441,285 parameters, 0 gradients, 3.3 GFLOPs
train: /content/dataset_split/train... found 12000 images in 5 classes ✅ 
val: /content/dataset_split/val... found 1500 images in 5 classes ✅ 
test: /content/dataset_split/test... found 1500 images in 5 classes ✅ 


               classes   top1_acc   top5_acc: 100%|██████████| 24/24 [00:04<00:00,  5.85it/s]


                   all      0.977          1
Speed: 0.1ms preprocess, 0.3ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to my_yolo_project/waste_classifier2


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ad4dd914d50>
curves: []
curves_results: []
fitness: 0.9886666536331177
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9773333072662354, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9886666536331177}
save_dir: PosixPath('my_yolo_project/waste_classifier2')
speed: {'preprocess': 0.06945254533366096, 'inference': 0.2688117539995195, 'loss': 0.004904519999399781, 'postprocess': 0.00036963599995942786}
task: 'classify'
top1: 0.9773333072662354
top5: 1.0

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
!mv /content/my_yolo_project/waste_classifier/weights/best.pt /content/Classi_YOLO.pt

# !cp /content/Classi_YOLO.pt /content/drive/MyDrive/


In [ ]:
# testing with test folder

from ultralytics import YOLO

model = YOLO('/content/Classi_YOLOv8.pt')

metrics = model.val(
    data='/content/dataset_split',
    split='test',
    imgsz=224,
    batch=32,
    device=0
)

# 4. In ra metrics
print(metrics)

print("Results saved in:", metrics.save_dir)

Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-cls summary (fused): 30 layers, 1,441,285 parameters, 0 gradients, 3.3 GFLOPs
train: /content/dataset_split/train... found 12000 images in 5 classes ✅ 
val: /content/dataset_split/val... found 1500 images in 5 classes ✅ 
test: /content/dataset_split/test... found 1500 images in 5 classes ✅ 
test: Fast image access ✅ (ping: 0.0±0.0 ms, read: 15.2±17.4 MB/s, size: 17.4 KB)


test: Scanning /content/dataset_split/test... 1500 images, 0 corrupt: 100%|██████████| 1500/1500 [00:01<00:00, 1171.27it/s]

test: New cache created: /content/dataset_split/test.cache



               classes   top1_acc   top5_acc:  15%|█▍        | 7/47 [00:01<00:05,  7.67it/s]

               classes   top1_acc   top5_acc:  23%|██▎       | 11/47 [00:01<00:04,  8.86it/s]
100%|██████████| 755k/755k [00:00<00:00, 34.6MB/s]
               classes   top1_acc   top5_acc: 100%|██████████| 47/47 [00:05<00:00,  8.86it/s]


                   all      0.994          1
Speed: 0.1ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/val
ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x78586810b990>
curves: []
curves_results: []
fitness: 0.9970000088214874
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9940000176429749, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9970000088214874}
save_dir: PosixPath('runs/classify/val')
speed: {'preprocess': 0.12421272333343344, 'inference': 0.5180688406668044, 'loss': 0.00040384866641337186, 'postprocess': 0.000710040000361308}
task: 'classify'
top1: 0.9940000176429749
top5: 1.0
Results saved in: runs/classify/val


# Training with MobileNet

In [ ]:
!pip install tensorflow

In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt

# Parameters
img_size = (224, 224)
batch_size = 32

# Load training data
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset_split/train',
    image_size=img_size,
    batch_size=batch_size
)

# Load validation data
val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset_split/val',
    image_size=img_size,
    batch_size=batch_size
)

# Load test data
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    'dataset_split/test',
    image_size=img_size,
    batch_size=batch_size
)

# Get number of classes automatically
class_names = train_dataset.class_names
num_classes = len(class_names)

# # Optimize performance
# AUTOTUNE = tf.data.AUTOTUNE
# train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
# val_dataset = val_dataset.prefetch(buffer_size=AUTOTUNE)
# test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)


Found 12000 files belonging to 5 classes.
Found 1500 files belonging to 5 classes.
Found 1500 files belonging to 5 classes.


In [ ]:
# Load MobileNetV2 without the top
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base
base_model.trainable = False

# Build model
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')])

# Compile
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           645 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,597 (9.24 MB)

 Trainable params: 164,613 (643.02 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
# Train
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=30
)


In [ ]:
model.save('classi_mobilenet.keras')

from google.colab import files
files.download('classi_mobilenet.keras')

model.save('classi_mobilenet.h5')

from google.colab import files
files.download('classi_mobilenet.h5')

In [ ]:
from tensorflow.keras.models import load_model

# Evaluate
model = load_model('/content/classi_mobilenet.h5')
test_loss, test_acc = model.evaluate(test_dataset)
print(f'Test accuracy: {test_acc:.3f}')


FailedPreconditionError: Graph execution error:

Detected at node StatefulPartitionedCall defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelbase.py", line 510, in dispatch_queue

  File "/usr/local/lib/python3.11/dist-pa

# Training EfficienNet

In [ ]:
!pip install efficientnet_pytorch

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from efficientnet_pytorch import EfficientNet
from torch.utils.data import DataLoader

In [ ]:
train_dir = '/content/dataset_split/train'
val_dir = '/content/dataset_split/val'
test_dir = '/content/dataset_split/test'

IMG_SIZE = 224
BATCH_SIZE = 32
NUM_EPOCHS = 30
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("GPU Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU Name:", torch.cuda.get_device_name(0))
else:
    print("no GPU")

GPU Available: True
GPU Name: Tesla T4


In [ ]:

# Transforms
train_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [ ]:
# 5. Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=train_transforms)
val_dataset = datasets.ImageFolder(val_dir, transform=val_transforms)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2) #true?
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2) #false?

print(f"🔹 Classes: {train_dataset.classes}")

# 6. Load EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b0')

# Update fc layer để phù hợp số class
num_classes = len(train_dataset.classes)
model._fc = nn.Linear(model._fc.in_features, num_classes)
model = model.to(DEVICE)

# 7. Loss và Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

# 8. Training loop
best_val_acc = 0.0

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Validation
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total
    print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] - Loss: {running_loss/len(train_loader):.4f} - Val Acc: {val_acc:.4f}")

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), '/content/efficientnet_b0_best.pt')
        print("✅ Best model saved!")

print("🎯 Training completed!")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
import os

# Device check
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")


Using device: cuda


In [ ]:
# Preprocess the images
transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

test_dir = '/content/dataset_split/test'
test_dataset = ImageFolder(test_dir, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [ ]:
import gdown

url = 'https://drive.google.com/uc?id=10s306Q3e1xM_S5HWRYjClOV_lYEdWpyH'
gdown.download(url, quiet=False)


In [ ]:
num_classes = len(test_dataset.classes)

model = EfficientNet.from_name('efficientnet-b0')
model._fc = torch.nn.Linear(model._fc.in_features, num_classes)

model_path = '/content/efficientnet_b0_best.pt'
model.load_state_dict(torch.load(model_path, map_location=device))
model = model.to(device)
model.eval()


EfficientNet(
  (_conv_stem): Conv2dStaticSamePadding(
    3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False
    (static_padding): ZeroPad2d((0, 1, 0, 1))
  )
  (_bn0): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
  (_blocks): ModuleList(
    (0): MBConvBlock(
      (_depthwise_conv): Conv2dStaticSamePadding(
        32, 32, kernel_size=(3, 3), stride=[1, 1], groups=32, bias=False
        (static_padding): ZeroPad2d((1, 1, 1, 1))
      )
      (_bn1): BatchNorm2d(32, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
      (_se_reduce): Conv2dStaticSamePadding(
        32, 8, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_se_expand): Conv2dStaticSamePadding(
        8, 32, kernel_size=(1, 1), stride=(1, 1)
        (static_padding): Identity()
      )
      (_project_conv): Conv2dStaticSamePadding(
        32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False
    

In [ ]:
import torch
import torch.nn as nn

# Define your loss function (assuming CrossEntropyLoss for classification)
criterion = nn.CrossEntropyLoss()

correct = 0
total = 0
test_loss = 0.0

# Set model to evaluation mode
model.eval()

# No gradients needed for validation/testing
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)

        # Calculate loss
        loss = criterion(outputs, labels)
        test_loss += loss.item()  # Accumulate the loss

        # Get predictions
        _, predicted = torch.max(outputs, 1)

        # Calculate accuracy
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Compute average loss over all batches
average_loss = test_loss / len(test_loader)

# Compute accuracy
accuracy = 100 * correct / total

# Print results
print(f'✅ Test Accuracy: {accuracy:.2f}%')
print(f'✅ Test Loss: {average_loss:.4f}')


✅ Test Accuracy: 97.53%
✅ Test Loss: 0.1154


In [ ]:
import os

# Create output directory if it doesn't exist
output_dir = '/content/output'
os.makedirs(output_dir, exist_ok=True)

# Example of your detection and classification code
for label in os.listdir(test_dir):
    label_dir = os.path.join(test_dir, label)
    for img_name in os.listdir(label_dir):
        img_path = os.path.join(label_dir, img_name)
        img = cv2.imread(img_path)

        # Run detection
        results = model(img)
        for result in results:
            if result.boxes is not None and result.boxes.cls.numel() > 0:
                # Use top-1 predicted box
                top_idx = result.boxes.conf.argmax()
                pred_class_idx = int(result.boxes.cls[top_idx].item())
                pred_class_name = result.names[pred_class_idx]

                # Compare with ground truth label (folder name)
                total_preds += 1
                if pred_class_name == label:
                    correct_preds += 1
                print(f"Image: {img_name}, Predicted: {pred_class_name}, Ground Truth: {label}")

                # Draw box and label
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, pred_class_name, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX,
                            0.9, (0, 255, 0), 2)

        # Save the processed image
        output_path = os.path.join(output_dir, f'{label}_{img_name}')
        cv2.imwrite(output_path, img)

print(f"✅ All results saved in: {output_dir}")
